In [7]:
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

import numpy as np 
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
#import jdc
from ExerciseAnswers import Answers

pylab.rcParams['figure.figsize'] = 13, 13

In this exercise you will implement three more methods in the `Drone2D` class: `z_dot_dot(self)`, `y_dot_dot(self)`, and `phi_dot_dot(self)`

Also, consider the following equations which we have seen in the previous videos:

$$ c = F_1 + F_2 $$
$$\ddot{z} = g - \cos(\phi) \frac{c}{m}$$

$$c = F_1 + F_2$$
$$\ddot{y} = \sin(\phi) \frac{c}{m}$$

$$M_x = L(F_1 - F_2)$$
$$\ddot{\phi} = \frac{M_x}{I_x}$$


In [16]:
class Drone2D:
    
    def __init__(self,
                 k_f = 0.1, # value of the thrust coefficient
                 I_x = 0.1, # moment of inertia around the x-axis
                 m = 1.0,   # mass of the vehicle 
                 l = 0.5,   # distance between the center of 
                            #   mass and the propeller axis
                ):
        
        # Initializing constants and input parameters
        self.k_f = k_f
        self.I_x = I_x
        self.l = l 
        self.m = m
        self.g = 9.81
        
        # Initial state
        self.omega_1 = 0.0
        self.omega_2 = 0.0
        self.X = np.array([0.0,0.0,0.0,0.0,0.0,0.0])   # z, y, phi, z_dot, y_dot, phi_dot
        
    def advance_state_uncontrolled(self,dt):
        """Advances the state of the drone by dt seconds. 
        Note that this method assumes zero rotational speed 
        for both propellers."""
        
        # Definition of X_dot
        # X_dot = z_dot, y_dot, phi_dot, z_dot_dot, y_dot_dot, phi_dot_dot        
        X_dot = np.array([self.X[3], self.X[4], self.X[5], self.g, 0.0, 0.0])
        
        # Change in state due to dt
        self.X += X_dot * dt
        
        return self.X
    
    def get_thrust_and_moment(self):
        """Helper function which calculates and returns the 
        collective thrust and the moment about the X axis"""

        # Thrust
        f1 = self.k_f * self.omega_1 ** 2   # propeller 1
        f2 = self.k_f * self.omega_2 ** 2   # propeller 2
        c = f1 + f2                         # "collective" thrust
        
        # Moment
        M_x = (f1 - f2) * self.l
        
        return c, M_x
    
    
    @property
    def z_dot_dot(self):
        """Vertical (z) acceleration of drone."""
        c, _ = self.get_thrust_and_moment()
        
        return (self.g - c * np.cos(self.X[2])) / self.m

    @property    
    def y_dot_dot(self):
        """Lateral (y) acceleration of drone."""
        c, _ = self.get_thrust_and_moment()
        
        return (+ c * np.sin(self.X[2])) / self.m
    
    @property
    def phi_dot_dot(self):
        ''' Angular acceleration about the x-axis (phi) of drone
        '''
        _, M_x = self.get_thrust_and_moment()
        
        return M_x / self.I_x

In [17]:
# TESTING CODE
# 
# run this cell to see how if your functions are working.

def equal_floats(num1, num2):
    return abs(num1 - num2) < 0.00001
 
drone = Drone2D(m=1.0, I_x=1.5, k_f=1.0)
drone.omega_1 = 1.0 
drone.omega_2 = 2.0

# tilt the test drone to 30 degrees
drone.X[2] = math.pi / 6

# test z_dot_dot
if equal_floats(drone.z_dot_dot, 5.47987):
    print("z_dot_dot works")
else:
    print("  ERROR in z_dot_dot")

# test y_dot_dot
if equal_floats(drone.y_dot_dot, 2.50):
    print("y_dot_dot works")
else:
    print("  ERROR in y_dot_dot")
    
# test y_dot_dot
if equal_floats(drone.phi_dot_dot, -1):
    print("phi_dot_dot works")
else:
    print("  ERROR in phi_dot_dot")

z_dot_dot works
y_dot_dot works
phi_dot_dot works
